In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import re
from time import sleep
import unicodedata
import os
from datetime import datetime
import locale

s = requests.Session()

senha = open(u'\\\\172.20.0.45\\jornalismo novo\_Utilit\xe1rios\password.txt').read()

proxy = ({
    'http':'http://rogerio.moreira:{}@172.20.0.75:8080'.format(senha),
	'https':'https://rogerio.moreira:{}@172.20.0.75:8080'.format(senha)
	})

def clip(df, **kwargs):
	return df.to_clipboard(float_format=CommaFloatFormatter(), sep='\t', **kwargs)

class CommaFloatFormatter:
	def __mod__(self, x):
		return str(x).replace('.',',')

def make_soup(url):
    html = s.get(url, proxies=proxy, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36'})
    soup = BeautifulSoup(html.content, "html5lib")
    return soup


locale.setlocale(locale.LC_TIME, "ptb")

site = 'http://www.iag.usp.br/astronomia/datas-de-mudanca-das-fases-da-lua'

os.chdir(r'C:\Users\rogerio.moreira\Desktop')

soup = make_soup(site)

In [5]:
"""
Quero uma tabela no formato:

dia (datetime) | lua | str para tempo
"""

def td(str_):
    if len(str_)<2 and str_!= '': str_='0'+str_
#     if re.match('^\d$',str_): str_='0'+str_
    return str_.title()

luas = ['Nova','Crescente','Cheia','Minguante']

l_ = []

for ano in [2018, 2019]:
    l = []

    tabela = soup.find('th', text=ano).parent.parent.parent

    linhas = tabela.findAll('tr')

    for linha in linhas:
        i = 0
        datas = ' '.join([td(el.text) for el in linha.findAll('td')]).split(' 0\xa0 ')
        l_datas = [el for el in datas if len(el)>2]
        if len(l_datas) > 0: l.extend(l_datas)

    i = 0
    if ano == 2018:
        i = 2

    for el in l:
        if el != '0\xa0 0\xa0':
            dt = datetime.strptime(el, '%b %d %H %M').replace(year=ano)
            dt_ = dt.strftime('%d de %B, %H:%M')
            l_.append([dt, luas[i], dt_])
            if i<3: i+=1
            else: i=0
        
os.chdir('S:\# Robôs\Tempo\# script')
        
pd.DataFrame(l_, columns = ['datetime', 'fase', 'texto']).to_csv('lua-datetime.csv', index=False, sep='\t')